##### Copyright 2018 The Autores de TensorFlow.

Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Mejor rendimiento con tf.function y AutoGraph

<table class="tfo-notebook-buttons" align="left">
  <td>
    <p data-md-type="paragraph"><a target="_blank" href="https://www.tensorflow.org/guide/function"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ver en TensorFlow.org</a></p>
  </td>
  <td>
    <p data-md-type="paragraph"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/function.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Ejecutar en Google Colab</a></p>
  </td>
  <td>
    <p data-md-type="paragraph"><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/function.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver código fuente en GitHub</a></p>
  </td>
  <td>
    <p data-md-type="paragraph"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/function.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Descargar cuaderno</a></p>
  </td>
</table>

TF 2.0 reúne la facilidad de ejecución ansiosa y el poder de TF 1.0. En el centro de esta fusión se encuentra `tf.function` , que le permite transformar un subconjunto de sintaxis de Python en gráficos portátiles TensorFlow de alto rendimiento.

Una característica nueva y genial de `tf.function` es AutoGraph, que le permite escribir código gráfico utilizando la sintaxis natural de Python. Para obtener una lista de las características de Python que puede usar con AutoGraph, consulte [Capacidades y limitaciones de AutoGraph](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md) . Para obtener más detalles sobre `tf.function` , consulte RFC [TF 2.0: funciones, no sesiones](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md) . Para obtener más detalles sobre AutoGraph, consulte `tf.autograph` .

Este tutorial lo guiará a través de las características básicas de `tf.function` y AutoGraph.

## Preparar

Importar TensorFlow 2.0:

In [0]:
import numpy as np

In [0]:
import tensorflow as tf

## El decorador `tf.function`

Cuando anota una función con `tf.function` , aún puede llamarla como cualquier otra función. Pero se compilará en un gráfico, lo que significa que obtendrá los beneficios de una ejecución más rápida, ejecutando en GPU o TPU, o exportando a SavedModel.

In [0]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

Si examinamos el resultado de la anotación, podemos ver que es un llamado especial que maneja todas las interacciones con el tiempo de ejecución de TensorFlow.

In [0]:
simple_nn_layer

Si su código usa múltiples funciones, no necesita anotarlas todas: cualquier función llamada desde una función anotada también se ejecutará en modo gráfico.

In [0]:
def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

Las funciones pueden ser más rápidas que el código entusiasta, para gráficos con muchas operaciones pequeñas. Pero para los gráficos con algunas operaciones costosas (como convoluciones), es posible que no vea mucha aceleración.

In [0]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")


In [0]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input, state); lstm_fn(input, state)
print("eager lstm:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("function lstm:", timeit.timeit(lambda: lstm_fn(input, state), number=10))


## Usar flujo de control de Python

Al usar el flujo de control dependiente de datos dentro de `tf.function` , puede usar las declaraciones de flujo de control de Python y AutoGraph las convertirá en operaciones apropiadas de TensorFlow. Por ejemplo, `if` declaraciones se convertirán en `tf.cond()` si dependen de un `Tensor` .

En el siguiente ejemplo, `x` es un `Tensor` pero la instrucción `if` funciona como se esperaba:

In [0]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

Nota: El ejemplo anterior usa condicionales simples con valores escalares. <a href="#batching">El procesamiento por lotes</a> generalmente se usa en el código del mundo real.

AutoGraph admite declaraciones comunes de Python como `while` , `for` , `if` , `break` , `continue` y `return` , con soporte para anidamiento. Eso significa que puede usar expresiones `Tensor` en la condición de las declaraciones `while` y `if` , o iterar sobre un `Tensor` en un bucle `for` .

In [0]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

AutoGraph también proporciona una API de bajo nivel para usuarios avanzados. Por ejemplo, podemos usarlo para echar un vistazo al código generado.

In [0]:
print(tf.autograph.to_code(sum_even.python_function))

Aquí hay un ejemplo de flujo de control más complicado:

In [0]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

## Keras y AutoGraph

AutoGraph está disponible por defecto en los modelos Keras no dinámicos. Para obtener más información, consulte `tf.keras` .

In [0]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

## Efectos secundarios

Al igual que en el modo ansioso, puede usar operaciones con efectos secundarios, como `tf.assign` o `tf.print` normalmente dentro de `tf.function` , e insertará las dependencias de control necesarias para garantizar que se ejecuten en orden.

In [0]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)


find_next_odd()
v

<a id="debugging"></a>

## Depuración

`tf.function` y AutoGraph funcionan generando código y trazándolo en gráficos TensorFlow. Este mecanismo aún no admite depuradores paso a paso como `pdb` . Sin embargo, puede llamar a `tf.config.experimental_run_functions_eagerly(True)` para habilitar temporalmente la ejecución ansiosa dentro de la `tf.function 'y usar su depurador favorito:

In [0]:
@tf.function
def f(x):
  if x > 0:
    # Try setting a breakpoint here!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

## Ejemplo avanzado: un ciclo de entrenamiento en el gráfico

La sección anterior mostró que AutoGraph se puede usar dentro de las capas y modelos de Keras. Los modelos Keras también se pueden usar en código AutoGraph.

Este ejemplo muestra cómo entrenar un modelo Keras simple en MNIST con todo el proceso de entrenamiento (carga de lotes, cálculo de gradientes, actualización de parámetros, cálculo de precisión de validación y repetición hasta la convergencia) se realiza en el gráfico.

### Descargar datos

In [0]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

### Define el modelo

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

### Definir el ciclo de entrenamiento.

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

## Procesamiento por lotes

En aplicaciones reales, el procesamiento por lotes es esencial para el rendimiento. El mejor código para convertir a AutoGraph es el código donde el flujo de control se decide a nivel de *lote* . Si toma decisiones a nivel de *ejemplo* individual, intente utilizar API por lotes para mantener el rendimiento.

Por ejemplo, si tiene el siguiente código en Python:

In [0]:
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]


square_if_positive(range(-5, 5))

Puede sentirse tentado a escribirlo en TensorFlow como tal (¡y esto funcionaría!):

In [0]:
@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack()


square_if_positive_naive(tf.range(-5, 5))

Pero en este caso, resulta que puedes escribir lo siguiente:

In [0]:
def square_if_positive_vectorized(x):
  return tf.where(x > 0, x ** 2, x)


square_if_positive_vectorized(tf.range(-5, 5))

## Re-rastreo

Puntos clave:

- Tenga cuidado al llamar a funciones con argumentos no tensoriales o con argumentos que cambian de forma.
- Decore funciones a nivel de módulo y métodos de clases a nivel de módulo, y evite decorar funciones o métodos locales.

`tf.function` puede darle una aceleración significativa sobre la ejecución ansiosa, a costa de una ejecución más lenta por primera vez. Esto se debe a que cuando se ejecuta por primera vez, la función también se *rastrea* en un gráfico TensorFlow. La construcción y optimización de un gráfico suele ser mucho más lenta en comparación con la ejecución real:

In [0]:
import timeit


@tf.function
def f(x, y):
  return tf.matmul(x, y)

print(
    "First invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

print(
    "Second invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

Puede saber fácilmente cuándo se rastrea una función agregando una declaración de `print` en la parte superior de la función. Debido a que cualquier código de Python solo se ejecuta en tiempo de rastreo, solo verá el resultado de `print` cuando se rastrea la función:

In [0]:
@tf.function
def f():
  print('Tracing!')
  tf.print('Executing')

print('First invocation:')
f()

print('Second invocation:')
f()

`tf.function` también puede *volver a rastrearse* cuando se llama con diferentes argumentos no tensoriales:

In [0]:
@tf.function
def f(n):
  print(n, 'Tracing!')
  tf.print(n, 'Executing')

f(1)
f(1)

f(2)
f(2)

Una *nueva traza* también puede ocurrir cuando los argumentos del tensor cambian de forma, a menos que haya especificado una `input_signature` :

In [0]:
@tf.function
def f(x):
  print(x.shape, 'Tracing!')
  tf.print(x, 'Executing')

f(tf.constant([1]))
f(tf.constant([2]))

f(tf.constant([1, 2]))
f(tf.constant([3, 4]))

Además, tf.function siempre crea una nueva función gráfica con su propio conjunto de trazas cada vez que se llama:

In [0]:
def f():
  print('Tracing!')
  tf.print('Executing')

tf.function(f)()
tf.function(f)()

Esto puede conducir a un comportamiento sorprendente cuando se usa el decorador `@tf.function` en una función anidada:

In [0]:
def outer():
  @tf.function
  def f():
    print('Tracing!')
    tf.print('Executing')
  f()

outer()
outer()